## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
data_dir = "ag_news"
inputdirectory = Path(f"./data/input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data/output/{out_dir}")

/tmp/ipykernel_761408/2673649470.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
ag_news = pd.read_csv("data/ag_news_train.csv", header=None)

In [3]:
# ag_news_txt = ag_news[2].str.cat(sep="\n")
ag_news_txt = ag_news[2].iloc[:50].str.cat(sep="\n")

In [4]:
with open("data/input/ag_news/ag_news.txt", "w") as f:
    f.write(ag_news_txt)

## Load Documents

In [5]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.09s/it]

Number of chunks =  7
Interest rates are trimmed to 7.5 by the South African central bank,  but the lack of warning hits the rand and surprises markets.

The cost of buying both new and second hand cars fell sharply over the past five years, a new survey has found.

South Korea's central bank cuts interest rates by a quarter percentage point to 3.5 in a bid to drive growth in the economy.

An auction of shares in Google, the web search engine which could be floated for as much as \$36bn, takes place on Friday.

Hewlett-Packard shares fall after disappointing third-quarter profits, while the firm warns the final quarter will also fall short of expectations.

One of the oldest textile operators on the Indian Ocean island of Mauritius last week shut seven factories and cut 900 jobs.

Chad asks the IMF for a loan to pay for looking after more than 100,000 refugees from conflict-torn Darfur in western Sudan.

The company running the Japanese nuclear plant hit by a fatal accident is to close

## Create a dataframe of all the chunks

In [6]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(7, 3)


/home/kleptotrace/miniforge3/envs/knowledge-graph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,text,source,chunk_id
0,"Reuters - Short-sellers, Wall Street's dwindli...",data/input/ag_news/ag_news.txt,a6d6a114fedc489e8e728f52442e132c
1,Forbes.com - After earning a PH.D. in Sociolog...,data/input/ag_news/ag_news.txt,17c75d86ec4a4cf6bd7528d8f66dde06
2,NEW YORK (Reuters) - The dollar tumbled broadl...,data/input/ag_news/ag_news.txt,f4e13a0503b041dbb14832734c0503cb
3,Interest rates are trimmed to 7.5 by the South...,data/input/ag_news/ag_news.txt,f58cdfcced924ea9a14b253aeaf857ad
4,A group led by the UAE's Etisalat plans to spe...,data/input/ag_news/ag_news.txt,924634f821b34f5dac95b6b3cbe6500e


## Extract Concepts

In [7]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [8]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='zephyr:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

100%|█████████████████████████████████████████████| 7/7 [00:34<00:00,  4.94s/it]

Possibly due to JSON Decode Error, 1 chunks have been skipped
(49, 5)


,node_1,node_2,edge,chunk_id,count
0,short-sellers,well-timed and occasionally controversial play...,"Carlyle Group, a private investment firm with ...",a6d6a114fedc489e8e728f52442e132c,4
1,oil prices,economy,Soaring crude prices plus worries about the ec...,a6d6a114fedc489e8e728f52442e132c,4
2,oil export flows,intelligence showed a rebel militia could stri...,Authorities have halted oil export flows from ...,a6d6a114fedc489e8e728f52442e132c,4
3,tearaway world oil prices,straining wallets,"Tearaway world oil prices, toppling records an...",a6d6a114fedc489e8e728f52442e132c,4
4,stocks,lowest for the year,Stocks ended slightly higher on Friday but sta...,a6d6a114fedc489e8e728f52442e132c,4


## Calculating contextual proximity

In [9]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
1371,us economy,underlings,"aff297e3bd6647a7bbb7e5227c3fa6d4,aff297e3bd664...",2,contextual proximity
1372,us economy,uzis and ak-47s,"aff297e3bd6647a7bbb7e5227c3fa6d4,aff297e3bd664...",2,contextual proximity
1382,uzis and ak-47s,microsoft,"aff297e3bd6647a7bbb7e5227c3fa6d4,aff297e3bd664...",2,contextual proximity
1389,uzis and ak-47s,us economy,"aff297e3bd6647a7bbb7e5227c3fa6d4,aff297e3bd664...",2,contextual proximity
1412,value-focused mutual funds,trade deficit,"f4e13a0503b041dbb14832734c0503cb,f4e13a0503b04...",3,contextual proximity


### Merge both the dataframes

In [10]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,abusers of legal weapons,microsoft,"aff297e3bd6647a7bbb7e5227c3fa6d4,aff297e3bd664...",contextual proximity,2
1,abusers of legal weapons,outrage,aff297e3bd6647a7bbb7e5227c3fa6d4,Target the abusers of legal weapons We can all...,4
2,abusers of legal weapons,us economy,"aff297e3bd6647a7bbb7e5227c3fa6d4,aff297e3bd664...",contextual proximity,2
3,assets of retail money market mutual funds,$849.98 trillion,a6d6a114fedc489e8e728f52442e132c,Assets of the nation's retail money market mut...,4
4,bidding,trade deficit,"f4e13a0503b041dbb14832734c0503cb,f4e13a0503b04...",contextual proximity,3
...,...,...,...,...,...
189,uzis and ak-47s,flooding the streets,aff297e3bd6647a7bbb7e5227c3fa6d4,Columnist Steve Bailey's argument that otherwi...,4
190,uzis and ak-47s,microsoft,"aff297e3bd6647a7bbb7e5227c3fa6d4,aff297e3bd664...",contextual proximity,2
191,uzis and ak-47s,us economy,"aff297e3bd6647a7bbb7e5227c3fa6d4,aff297e3bd664...",contextual proximity,2
192,value-focused mutual funds,losses,f4e13a0503b041dbb14832734c0503cb,Investors in value-focused mutual funds have r...,4


## Calculate the NetworkX Graph

In [11]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(87,)

In [12]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [13]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  17
[['$849.98 trillion', 'assets of retail money market mutual funds'], ['abusers of legal weapons', 'bill gates', 'blunder', 'competing business', 'flooding the streets', 'gateway artisan block', 'growth', 'improvements', 'marlborough-based technology company', 'microsoft', 'outrage', 'parceled out praise', 'temper', 'turned the corner', 'u-turn', 'underlings', 'us economy', 'uzis and ak-47s'], ['bidding', 'dollar', 'drawing foreign capital', 'economic growth', 'economic recovery', 'eurozone economy', "google's public offering", 'imports', 'interest rates', 'japan', "kids'", 'last-minute worries', 'later in the year', 'losses', 'oil costs', 'playboy magazine', 'purchasing power', 'shell', 'slowing down', 'south african central bank', 'takeover attempt', 'trade deficit', 'value-focused mutual funds'], ['central square in lynn', 'dell', 'gary winnick', 'global crossing', 'government', 'kevin b. rollins', 'large loss', 'lynn', 'nobility', 'quality distribution', 

### Create a dataframe for community colors

In [14]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows.append({"node": node, "color": color, "group": group})
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,$849.98 trillion,#ad57db,1
1,assets of retail money market mutual funds,#ad57db,1
2,abusers of legal weapons,#db5f57,2
3,bill gates,#db5f57,2
4,blunder,#db5f57,2
...,...,...,...
82,specialist indian engineers,#db577e,15
83,short-sellers,#57db9d,16
84,well-timed and occasionally controversial play...,#57db9d,16
85,straining wallets,#db8d57,17


### Add colors to the graph

In [15]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [16]:
from pyvis.network import Network

graph_output_directory = "../graphs/graph.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory)

                                       
[Select a Node by ID                                                    ]
Reset Selection
